### Predicting whether a given blight ticket will be paid on time, based on previous observed data


### - Data Visualization 

In [2]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv('dataset/train.csv', encoding="ISO-8859-1")
test = pd.read_csv('dataset/test.csv',encoding='ISO-8859-1')
addresses = pd.read_csv('dataset/addresses.csv')
latlons = pd.read_csv('dataset/latlons.csv')

In [3]:
train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 34 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ticket_id                   250306 non-null  int64  
 1   agency_name                 250306 non-null  object 
 2   inspector_name              250306 non-null  object 
 3   violator_name               250272 non-null  object 
 4   violation_street_number     250306 non-null  float64
 5   violation_street_name       250306 non-null  object 
 6   violation_zip_code          0 non-null       float64
 7   mailing_address_str_number  246704 non-null  float64
 8   mailing_address_str_name    250302 non-null  object 
 9   city                        250306 non-null  object 
 10  state                       250213 non-null  object 
 11  zip_code                    250305 non-null  object 
 12  non_us_str_code             3 non-null       object 
 13  country       

In [5]:
addresses.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [6]:
latlons.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


###  - Data pre-processing 
#### Pre-processing steps
 - Droping all rows with compliance value NuLL
 - Droping all rows not relevant to the the U.S
 - Merging latlons and addresses with the main data
 - Droping all unnecessary columns

In [7]:
train = train[np.isfinite(train['compliance'])]

In [8]:
train = train[train.country == 'USA']
test = test[test.country == 'USA']

In [9]:
train = pd.merge(train, pd.merge(addresses, latlons, on='address'), on='ticket_id')
test = pd.merge(test, pd.merge(addresses, latlons, on='address'), on='ticket_id')


In [10]:
train.drop(['agency_name', 'inspector_name', 'violator_name', 'non_us_str_code', 'violation_description',
                'grafitti_status', 'state_fee', 'admin_fee', 'ticket_issued_date', 'hearing_date',
                # columns not available in test data
                'payment_amount', 'balance_due', 'payment_date', 'payment_status',
                'collection_status', 'compliance_detail',
                # address related columns
                'violation_zip_code', 'country', 'address', 'violation_street_number',
                'violation_street_name', 'mailing_address_str_number', 'mailing_address_str_name',
                'city', 'state', 'zip_code', 'address'], axis=1, inplace=True)

In [11]:
train.head()

,ticket_id,violation_code,disposition,fine_amount,late_fee,discount_amount,clean_up_cost,judgment_amount,compliance,lat,lon
0,22056,9-1-36(a),Responsible by Default,250.0,25.0,0.0,0.0,305.0,0.0,42.390729,-83.124268
1,27586,61-63.0600,Responsible by Determination,750.0,75.0,0.0,0.0,855.0,1.0,42.326937,-83.135118
2,22046,9-1-36(a),Responsible by Default,250.0,25.0,0.0,0.0,305.0,0.0,42.145257,-83.208233
3,18738,61-63.0500,Responsible by Default,750.0,75.0,0.0,0.0,855.0,0.0,42.433466,-83.023493
4,18735,61-63.0100,Responsible by Default,100.0,10.0,0.0,0.0,140.0,0.0,42.388641,-83.037858


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159869 entries, 0 to 159868
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ticket_id        159869 non-null  int64  
 1   violation_code   159869 non-null  object 
 2   disposition      159869 non-null  object 
 3   fine_amount      159869 non-null  float64
 4   late_fee         159869 non-null  float64
 5   discount_amount  159869 non-null  float64
 6   clean_up_cost    159869 non-null  float64
 7   judgment_amount  159869 non-null  float64
 8   compliance       159869 non-null  float64
 9   lat              159867 non-null  float64
 10  lon              159867 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 14.6+ MB


#### Encoding discrete data columns

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
label_encoder = LabelEncoder()
label_encoder.fit(train['disposition'].append(test['disposition'], ignore_index=True))
train['disposition'] = label_encoder.transform(train['disposition'])
test['disposition'] = label_encoder.transform(test['disposition'])

In [15]:
label_encoder = LabelEncoder()
label_encoder.fit(train['violation_code'].append(test['violation_code'], ignore_index=True))
train['violation_code'] = label_encoder.transform(train['violation_code'])
test['violation_code'] = label_encoder.transform(test['violation_code'])

#### Adding missing values 

In [16]:
train['lat'] = train['lat'].fillna(train['lat'].mean())
train['lon'] = train['lon'].fillna(train['lon'].mean())

In [17]:
test['lat'] = test['lat'].fillna(test['lat'].mean())
test['lon'] = test['lon'].fillna(test['lon'].mean())

#### Defining Training and Testing Data

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train_columns = list(train.columns.values)
train_columns.remove('compliance')
test = test[train_columns]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(train.iloc[:, train.columns != 'compliance'], train['compliance'])

### - Defining the Model

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

In [22]:
regr_rf = RandomForestRegressor()
grid_values = {'n_estimators': [10, 100], 'max_depth': [None, 30]}
grid_clf_auc = GridSearchCV(regr_rf, param_grid=grid_values, scoring='roc_auc')
grid_clf_auc.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n

### - Model Evaluation 

In [23]:
from sklearn.metrics import roc_auc_score

In [24]:
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

Grid best parameter (max. AUC):  {'max_depth': 30, 'n_estimators': 100}
Grid best score (AUC):  0.8103109936392048


### - Model Predictions

In [25]:
pd.DataFrame(grid_clf_auc.predict(X_test), X_test.ticket_id, columns=['compliance'])

,compliance
ticket_id,
35753,0.243333
276319,0.082500
236688,0.000038
149595,0.000142
257927,0.003229
...,...
253769,0.171692
124356,0.071444
269017,0.000000
